In [1]:
from netmiko import ConnectHandler
%matplotlib inline
import pandas, matplotlib
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.lines import Line2D 
from pandas import Series, DataFrame, Panel
from IPython.display import display
import ipywidgets as widgets
from io import StringIO
from pandas.tools.plotting import table

import numpy as np
import time

df1 = pandas.read_csv('Info.txt', names = ["Equipo","IP","User","Pass"])
df2 = pandas.read_csv('Commands.txt', names = ["Comando"])

In [2]:
command_list = df2.to_csv(None, header=False, index=False).split('\n')
def f(command_str): 
  command_list = command_str.split(',')
  return [float(command_list[0]), int(command_list[1])]
command_list=command_list[:-1]

device_list = df1['Equipo'].to_csv(None, header=False, index=False).split('\n')
def f(device_str): 
  device_list = device_str.split(',')
  return [float(device_list[0]), int(device_list[1])]
device_list=device_list[:-1]

ip_list = df1['IP'].to_csv(None, header=False, index=False).split('\n')
def f(ip_str): 
  ip_list = ip_str.split(',')
  return [float(ip_list[0]), int(ip_list[1])]
ip_list=ip_list[:-1]

user_list = df1['User'].to_csv(None, header=False, index=False).split('\n')
def f(user_str): 
  user_list = user_str.split(',')
  return [float(user_list[0]), int(user_list[1])]
user_list=user_list[:-1]

pass_list = df1['Pass'].to_csv(None, header=False, index=False).split('\n')
def f(pass_str): 
  pass_list = pass_str.split(',')
  return [float(pass_list[0]), int(pass_list[1])]
pass_list=pass_list[:-1]

#command_list

In [3]:
a=0
for j in device_list:
    # Create a dictionary representing the device.
    cisco_881 = {
    'device_type': 'cisco_ios',
    'ip': ip_list[a],
    'username': user_list[a],
    'password': pass_list[a],
    #'port' : 8022, # optional, defaults to 22
    #'secret': 'secret', # optional, defaults to ''
    #'verbose': False, # optional, defaults to True
    }
    equi=device_list[a]
    net_connect = ConnectHandler(**cisco_881)
    today = datetime.today().strftime("%y%m%d")
    for row in command_list:
        filename=row
        filename = filename.replace('/', '')
        time.sleep(2)
        output = net_connect.send_command(str(row))
        text_file = open(equi+'_'+filename+'.txt', "w")
        text_file.write(output)
        text_file.close()
    
    
    a=a+1


#print (output)

In [50]:
a=0
v=0
dict1 = ["===","====","====","====","====","====","===="]
for j in device_list:
    equi=j
    s = StringIO()
    with open(equi+'_show crash list.txt') as f:
        for line in f:
            v=v+1
            if v is 1:
                fecha=line
            elif not (line.startswith(' ') or line.startswith('=')):
                while '  ' in line:
                    line = line.replace('  ', ' ')
                s.write(line)
    s.seek(0) # "rewind" to the beginning of the StringIO object
    h= pandas.read_csv(s, names = ["#","Time", "Process","Card/CPU/PID","SW Version","HW_SER_NUM"], sep=' ')
    #h.columns=h[:1]
    if (h[-1:].Time == "crash").item():
        h[-1:]=["No crash","No crash","No crash","No crash","No crash","No crash"]
        f=h[-1:]
        
    else:
        f=h[-3:-1:]
        
    f.insert(0, 'EQUIPO', equi)
    f.loc[-1]=dict1
    #print(f)
    if a is 0:
        a=a+1
        result=f.copy()
    else: 
        
        result1 = result.append(f, ignore_index=True)
        result=result1
    
result

c:\python\lib\site-packages\ipykernel\__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


'Wednesday July 20 12:12:18 BOT 2016\n'

In [48]:
dict1 = ["===","====","====","===="]
a=0
for j in device_list:
    equi=j
    s = StringIO()
    with open(equi+'_show session disconnect-reasons.txt') as f:
        for line in f:
            if not (line.startswith(' ') or line.startswith('=')):
                while '   ' in line:
                    line = line.replace('   ', '  ')
                s.write(line)
    s.seek(0)#ewind" to the beginning of the StringIO object
    h= pandas.read_csv(s, skiprows=5, sep='  ')
    j=(h.groupby(['Disconnect Reason'], as_index=False).sum()).sort('Percentage', ascending=False)[:5]
    j.insert(0, 'EQUIPO', equi)
    j.loc[-1]=dict1
    result.append(dict1, ignore_index=True)
    #print(f)
    if a is 0:
        a=a+1
        result=j.copy()
    else: 
        result1 = result.append(j, ignore_index=True)
        result=result1
    
result
j

c:\python\lib\site-packages\ipykernel\__main__.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
c:\python\lib\site-packages\ipykernel\__main__.py:14: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
c:\python\lib\site-packages\pandas\indexes\api.py:71: RuntimeWarning: unorderable types: str() < int(), sort order is undefined for incomparable objects
  result = result.union(other)


,EQUIPO,Disconnect Reason,Num Disc,Percentage
11,MONGGSN1,Remote-disconnect,185900180,90.66244
8,MONGGSN1,Idle-Inactivity-timeout,11004447,5.36680
4,MONGGSN1,Auth-failed,6289127,3.06717
17,MONGGSN1,failed-auth-with-charging-svc,445439,0.21724
3,MONGGSN1,Admin-disconnect,408670,0.19931
-1,===,====,====,====


In [68]:
dict1 = ["===","====","====","===="]
a=0
for j in device_list:
    equi=j
    s = StringIO()
    with open(equi+'_show context all.txt') as f:
        for line in f:
            if not (line.startswith(' ') or line.startswith('-')):
                while '  ' in line:
                    line = line.replace('  ', ' ')
                s.write(line)
    s.seek(0)#ewind" to the beginning of the StringIO object
    h= pandas.read_csv(s, header=1, sep=' ')
    #j=(h.groupby(['Disconnect Reason'], as_index=False).sum()).sort('Percentage', ascending=False)[:5]
    h.columns=["Context Name","ContextID", "State","D","DA"]
    j=h[['Context Name','ContextID', 'State']]
    j.insert(0, 'EQUIPO', equi)
    j.loc[-1]=dict1
    #result.append(dict1, ignore_index=True)
    #print(f)
    if a is 0:
        a=a+1
        result=j.copy()
    else: 
        result1 = result.append(j, ignore_index=True)
        result=result1
    
result

c:\python\lib\site-packages\ipykernel\__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,EQUIPO,Context Name,ContextID,State
0,ASU-ASR5K5-1,local,1,Active
1,ASU-ASR5K5-1,Pgw,2,Active
2,ASU-ASR5K5-1,Gi,3,Active
3,ASU-ASR5K5-1,Ga,4,Active
4,ASU-ASR5K5-1,Gi-VVM,5,Active
5,ASU-ASR5K5-1,G,6,Active
6,===,====,====,====
7,COR-ASR5K-1,local,1,Active
8,COR-ASR5K-1,Pgw,2,Active
9,COR-ASR5K-1,Gi,3,Active


In [291]:
index1= ["ASU-ASR5K5-1","COR-ASR5K-1","JO2-ASR5K5-1","ROS-ASR5K-1","TOR-ASR5K5-1","ASUGGSN1","MONGGSN1"]

val = widgets.RadioButtons(
    options=index1,
#    value='2',
    description='Contador:',
    )
display(val)

In [296]:
i=val.value
s = StringIO()
s.write("Endpoint")
a=0
b=0
with open(i+'_show diameter peer full all.txt') as f:
    for line in f:
        if (line.startswith('Context:') or (a==0)):
            s = StringIO()
            a=1
            while '  ' in line:
                line = line.replace('  ', ' ')
            line = line.replace('Context:', '')
            line = line.replace('Endpoint:', '-')
            line = line.replace('\n', '')
            context=line
            s.write(line)
        elif (line.startswith(' Peer Hostname:')):
            line = line.replace('Peer Hostname: ', '')
            line = line.replace('\n', '')
            s.write(line)
            peer=line
        elif (line.startswith('  Local Hostname:')):
            line = line.replace('Local Hostname: ', '')
            line = line.replace('\n', '')
            s.write(line)
            host=line
        elif (line.startswith('  State:')):
            line = line.replace('State:', '')
            line = line.replace('[TCP]', '')
            line = line.replace('\n', '')
            s.write(line)
            state=line
        elif (line.startswith('  CPU:')):
            line = line.replace('CPU:', '')
            line = line.replace('Task:', '-')
            line = line.replace('\n', '')
            s.write(line)
            cpu=line
        elif (line.startswith('  Peer Backoff Timer running:')):
            s.seek(0)#ewind" to the beginning of the StringIO object
            h= pandas.read_csv(s)
            h1=h.T
            #print(h1)
            h1.insert(0,'CPU - Task', cpu)
            h1.insert(0,'State', state)
            h1.insert(0,'Peer Hostname', host)
            h1.insert(0,'Local Hostname', peer)
            h1.insert(0,'Context - Endpoint', context)
            h1.insert(0, 'EQUIPO', i)
            #a=0
            s = StringIO()
            s.seek(0)
            s.write(context)
            #h1.columns=["context","peer","local","state","CPU","a","b","c","d"]
            #print(h1)
            if b is 0:
                b=1
                result=h1.copy()
            else: 
                result1 = result.append(h1, ignore_index=True)
                result=result1
#h.columns=["Context Name/Endpoint"]
result2=(result.sort(['State','Context - Endpoint','Local Hostname' ], ascending=True))
result4=result2.copy()
result4['Total Peers']=1
result3=(result4.groupby(['State'], as_index=False).sum()).sort('State', ascending=False)
result3

c:\python\lib\site-packages\ipykernel\__main__.py:64: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
c:\python\lib\site-packages\ipykernel\__main__.py:67: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,State,Total Peers
2,WAIT_CONNECT,18
1,OPEN,768
0,IDLE,6


In [71]:
#i=sm3.value
s = StringIO()
dict1 = ["===","===="]
b=0
result=[]
result1=[]
result3=[]
resultfinal=[]
for j in device_list:
    equi=j
    s = StringIO()
    with open(j+'_show snmp trap history verbose.txt') as f:
        for line in f:
            if not (line.startswith(' ') or line.startswith('-')):
                #line = line.replace('Internal', '|')
                #line = line.replace(') ', '|')
                lin=line[line.find("(")+1:line.find(")")]
                #lin1=line[line.find(")")+1:]
                #lin2=line[:line.find("Internal")]
                totaline= [equi,lin]
                #print (totaline)
                b=b+1
                
                if b is 5:
                    h=pandas.DataFrame(totaline)
                    h1=h.T
                    result=h1.copy()
                    #result.loc[-1]=dict1
                elif b > 5:
                    h=pandas.DataFrame(totaline)
                    h1=h.T
                    result1 = result.append(h1, ignore_index=False)
                    result=result1
                    #result.loc[-1]=dict1
result.columns=["Device","Facility"]
resultfinal=result.drop_duplicates(subset=['Device','Facility'])
resultfinal

,Device,Facility
0,ASU-ASR5K5-1,CLISessStart
0,ASU-ASR5K5-1,CLISessEnd
0,ASU-ASR5K5-1,LoginFailure
0,ASU-ASR5K5-1,PortDown
0,ASU-ASR5K5-1,PortLinkDown
0,ASU-ASR5K5-1,CiscoFruCardStatusChanged
0,ASU-ASR5K5-1,CardDown
0,ASU-ASR5K5-1,EntStateOperDisabled
0,ASU-ASR5K5-1,CardRemoved
0,ASU-ASR5K5-1,CiscoFruRemoved


In [11]:
index1=device_list

sm3 = widgets.RadioButtons(
    options=index1,
#    value='2',
    description='Contador:',
    )
display(sm3)

In [12]:
i=sm3.value
s = StringIO()
b=0
result=[]
result1=[]
with open(i+'_show snmp trap history verbose.txt') as f:
    equi=i
    for line in f:
        if not (line.startswith(' ') or line.startswith('-')):
            #line = line.replace('Internal', '|')
            #line = line.replace(') ', '|')
            lin=line[line.find("(")+1:line.find(")")]
            lin1=line[line.find(")")+1:]
            lin2=line[:line.find("Internal")]
            totaline= [equi,lin2,lin,lin1]
            #print (totaline)
            b=b+1
            if b is 5:
                h=pandas.DataFrame(totaline)
                h1=h.T
                result=h1.copy()
            elif b > 5:
                h=pandas.DataFrame(totaline)
                h1=h.T
                result1 = result.append(h1, ignore_index=False)
                result=result1
result.columns=["Equipo","Time","Facility","Description"]

index1=result['Facility'].drop_duplicates().values.tolist()

sm2 = widgets.SelectMultiple(
    options=index1,
    description='Warning:')
display(sm2)

In [13]:
resulttotal=result[result['Facility'].isin(sm2.value)]
resulttotal

,Equipo,Time,Facility,Description
0,COR-ASR5K-1,Wed Jul 20 13:35:09 2016,MemoryOver,facility cli instance 8020062 card 8 cpu 0 al...
0,COR-ASR5K-1,Wed Jul 20 13:35:19 2016,MemoryOverClear,facility cli instance 8020062 card 8 cpu 0 al...
0,COR-ASR5K-1,Fri Jul 22 04:54:09 2016,MemoryWarn,facility cli instance 8020062 card 8 cpu 0 al...
0,COR-ASR5K-1,Fri Jul 22 04:54:19 2016,MemoryWarnClear,facility cli instance 8020062 card 8 cpu 0 al...
0,COR-ASR5K-1,Mon Jul 25 13:05:59 2016,MemoryOver,facility npumgr instance 8 card 8 cpu 0 alloc...
0,COR-ASR5K-1,Mon Jul 25 13:06:09 2016,MemoryOverClear,facility npumgr instance 8 card 8 cpu 0 alloc...
0,COR-ASR5K-1,Mon Jul 25 17:47:29 2016,CPUWarn,facility cli instance 8003184 card 8 cpu 0 al...
0,COR-ASR5K-1,Mon Jul 25 17:48:29 2016,CPUWarnClear,facility cli instance 8003184 card 8 cpu 0 al...
0,COR-ASR5K-1,Mon Jul 25 17:50:49 2016,MemoryWarn,facility cli instance 8003184 card 8 cpu 0 al...
0,COR-ASR5K-1,Mon Jul 25 17:50:59 2016,MemoryWarnClear,facility cli instance 8003184 card 8 cpu 0 al...


In [55]:
a=0
s = StringIO()
b=0
result=[]
result1=[]
for j in device_list:
    equi=j
    s = StringIO()
    with open(equi+'_show version.txt') as f:
        for line in f:
            if (line.startswith('  Image Version')):
                line = line.replace('  Image Version:                 ', '')
                line = line.replace('\n', '')
                s.write(line)
                version=line
            elif (line.startswith('  Boot Image')):
                s = StringIO()
                s.seek(0)
                a=1
                line = line.replace('  Boot Image:                   ', '')
                line = line.replace('\n', '')
                s.write(line)
                boot=line
                totaline= [equi,version,boot]
                b=b+1
                if b < 2:
                    h=pandas.DataFrame(totaline)
                    h1=h.T
                    result=h1.copy()
                else :
                    h=pandas.DataFrame(totaline)
                    h1=h.T
                    result1 = result.append(h1, ignore_index=False)
                    result=result1
result.columns=["Device","Version","Boot Image"]
result

,Device,Version,Boot Image
0,ASU-ASR5K5-1,17.3.2,/flash/production.60474.asr5500.bin
0,COR-ASR5K-1,17.3.2,/flash/production.60474.asr5000.bin
0,JO2-ASR5K5-1,19.3.2,/flash/production.63505.asr5500.bin
0,ROS-ASR5K-1,17.3.2,/flash/production.60474.asr5000.bin
0,TOR-ASR5K5-1,17.3.2,/flash/production.60474.asr5500.bin
0,ASUGGSN1,17.1.0,/flash/production.58241.asr5000.bin
0,MONGGSN1,17.1.0,/flash/production.58241.asr5000.bin


In [41]:
# result=[]
dict1 = ["===","====","====","====","===","====","====","====","===="]
b=0
for j in device_list:
    equi=j
    path = '_show session recovery status verbose.txt'
    col_specification =[(1, 5), (5, 13), (14, 21), (21, 29), (30, 37), (37, 45), (46, 53), (54, 80),]
    data = pandas.read_fwf(equi+path, colspecs=col_specification)
    h1=data.iloc[7:]   
    h1.columns=data.loc[5]
    h1.insert(0, 'EQUIPO', j)
    h1.loc[-1]=dict1
    b=b+1
    if b < 2:
        #h=pandas.DataFrame(totaline)
        #h1=h.T
        result=h1.copy()
    else :
        #h=pandas.DataFrame(totaline)
        #h1=h.T
        result1 = result.append(h1, ignore_index=False)
        result=result1
#result.columns=["Device","Version","Boot Image"]
result

c:\python\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


5,EQUIPO,cpu,state,active,standby,active,standby,active,status
7,ASU-ASR5K5-1,1/0,Standby,0,24,0,25,0,Good
8,ASU-ASR5K5-1,1/1,Standby,0,24,0,24,0,Good
9,ASU-ASR5K5-1,2/0,Active,24,1,24,1,0,Good
10,ASU-ASR5K5-1,2/1,Active,24,1,24,1,0,Good
11,ASU-ASR5K5-1,3/0,Active,0,0,0,0,3,Good (Demux)
12,ASU-ASR5K5-1,3/1,Active,0,0,0,0,5,Good (Demux)
13,ASU-ASR5K5-1,8/0,Active,24,1,25,1,0,Good
14,ASU-ASR5K5-1,8/1,Active,24,1,23,1,0,Good
15,ASU-ASR5K5-1,9/0,Active,24,1,24,1,0,Good
16,ASU-ASR5K5-1,9/1,Active,24,1,24,1,0,Good


In [37]:
b=0 
j=0
a=0
dict1 = ["===","====","====","====","===","==="]
for d in device_list:
    equi=d
    s = StringIO()
    b=0
    j=0
    n=0
    with open(equi+'_show alarm outstanding verbose.txt') as f:
        for line in f:
            j=j+1
            if (line.startswith('Informational') or line.startswith('Minor') or line.startswith('Major') or line.startswith('Warning') or line.startswith('Critical')):
                s.write(line)
                col_names = ['Severity','Objects','Timestamp','Alarm ID']
                col_specification =[(0, 8), (9, 25), (25, 59), (60, 80),]
                data = pandas.read_fwf(equi+'_show alarm outstanding verbose.txt', colspecs=col_specification , nrows=1, skiprows=j-1, names=col_names)
                b=1
            elif b is 1:
                col_names = ['Details']
                col_specification =[(0, 80),]
                data1 = pandas.read_fwf(equi+'_show alarm outstanding verbose.txt', colspecs=col_specification , nrows=1, skiprows=j-1, names=col_names)
                b=0
                data.insert(4, 'Description', data1)
                data.insert(0, 'Equipo', equi)
                if a is 0:
                    
                    result=data.copy()
                    n=1
                    a=1
                else :
                    
                    result1 = result.append(data, ignore_index=False)
                    result=result1
                    n=1
    
result

,Equipo,Severity,Objects,Timestamp,Alarm ID,Description
0,COR-ASR5K-1,Minor,Card 26,Friday May 06 15:41:11 ART,5791100448530694144,The 10 Gig Ethernet Line Card in slot 26 is a ...
0,COR-ASR5K-1,Minor,Card 27,Friday May 06 15:41:26 ART,5791100448531677184,The 10 Gig Ethernet Line Card in slot 27 is a ...
0,COR-ASR5K-1,Minor,Card 28,Friday May 06 15:41:26 ART,5791100448531677185,The 10 Gig Ethernet Line Card in slot 28 is a ...
0,COR-ASR5K-1,Minor,Card 30,Friday May 06 15:41:26 ART,5791100448531677187,The 10 Gig Ethernet Line Card in slot 30 is a ...
0,COR-ASR5K-1,Minor,Card 31,Friday May 06 15:41:26 ART,5791100448531677188,The 10 Gig Ethernet Line Card in slot 31 is a ...
0,COR-ASR5K-1,Minor,Card 29,Wednesday May 25 00:23:08 A,5791100552504672256,The 10 Gig Ethernet Line Card in slot 29 is a ...
0,ROS-ASR5K-1,Minor,Card 18,Monday October 05 15:31:13,5771177185573404672,The 10 Gig Ethernet Line Card in slot 18 is a ...
0,ROS-ASR5K-1,Minor,Card 19,Monday October 05 15:31:13,5771177185573404673,The 10 Gig Ethernet Line Card in slot 19 is a ...
0,ROS-ASR5K-1,Minor,Card 20,Monday October 05 15:31:13,5771177185573404674,The 10 Gig Ethernet Line Card in slot 20 is a ...
0,ROS-ASR5K-1,Minor,Card 22,Monday October 05 15:31:13,5771177185573404676,The 10 Gig Ethernet Line Card in slot 22 is a ...


In [52]:
b=0
dict1 = ["===","====","====","====","===","==="]
for j in device_list:
    equi=j
    path = '_show card table.txt'
    col_specification =[(0, 11), (13, 52), (53, 67), (68, 73), (75, 81),]
    col_names = ['Slot','Card Type','Oper State','SPOF','Attach']
    data = pandas.read_fwf(equi+path, colspecs=col_specification, names=col_names)
    h1=data.iloc[3:]   
    h1.insert(0, 'EQUIPO', j)
    b=b+1
    if b < 2:
        #h=pandas.DataFrame(totaline)
        #h1=h.T
        h1.loc[-1]=dict1
        result=h1.copy()
    else :
        #h=pandas.DataFrame(totaline)
        #h1=h.T
        h1.loc[-1]=dict1
        result1 = result.append(h1, ignore_index=False)
        result=result1
        
#result.columns=["Device","Version","Boot Image"]
result
filtered = result[(result['Card Type']!='None')]
filtered

c:\python\lib\site-packages\ipykernel\__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\python\lib\site-packages\ipykernel\__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,EQUIPO,Slot,Card Type,Oper State,SPOF,Attach
3,ASU-ASR5K5-1,1: DPC,Universal Data Processing Card,Standby,-,NaN
4,ASU-ASR5K5-1,2: DPC,Universal Data Processing Card,Active,No,NaN
5,ASU-ASR5K5-1,3: DPC,Universal Data Processing Card,Active,No,NaN
7,ASU-ASR5K5-1,5: MMIO,Universal Mgmt & 20x10Gb I/O Card,Active,No,NaN
8,ASU-ASR5K5-1,6: MMIO,Universal Mgmt & 20x10Gb I/O Card,Standby,-,NaN
10,ASU-ASR5K5-1,8: DPC,Universal Data Processing Card,Active,No,NaN
11,ASU-ASR5K5-1,9: DPC,Universal Data Processing Card,Active,No,NaN
13,ASU-ASR5K5-1,11: SSC,System Status Card,Active,No,NaN
14,ASU-ASR5K5-1,12: SSC,System Status Card,Active,No,NaN
16,ASU-ASR5K5-1,14: FSC,Fabric & 2x200GB Storage Card,Active,No,NaN


In [286]:
dict1 = ["===","====","====","====","===","===","==="]
b=0
for u in device_list:
    equi=u
    
    with open(equi+'_show resources.txt') as f:
            for line in f:
                if (line.startswith('    ')):
                    if (line.startswith('    In Use')):
                        line1 = line.replace('    In Use              : ', '')
                        line1 = line1.replace('\n', '')
                        line1 = line1.replace(' ', '')
                        pandas.to_numeric(line1, errors='coerce')
                        a=1

                    elif (line.startswith('    Max Used') & (a is 1)):
                        line2 = line.replace('    Max Used            : ', '')
                        line2 = line2.replace('\n', '')
                        line2 = line2.replace(' ', '')
                        #pandas.to_numeric(line2, errors='coerce')
                        d=1
                    elif (line.startswith('    Limit') & (d is 1)):
                        line3 = line.replace('    Limit               : ', '')
                        line3 = line3.replace('\n', '')
                        line3 = line3.replace(' ', '')
                        pandas.to_numeric(line3, errors='coerce')
                        f=1
                    if f is 1:
                        l4=int(line1)
                        l5=int(line3)
                        l6=((l4/l5)*100)
                        l7=int(l6)
                        totaline= [line1,line2,line3,l7,equi]
                        h=pandas.DataFrame(totaline)
                        h1=h.T
                        h1.colums=["InUse","MaxUsed","Limit","Utilization","Equipo"]
                        f=0
                        resu=[]
                        resu = pandas.concat([h3, h1], axis=1)
                        resu.columns=["Service","Type","InUse","MaxUsed","Limit","Utilization","Equipo"]
                        #print(h3)
                        #print(h1)
                        if b is 0:
                            b=b+1
                            result=resu.copy()
                        else :
                            result1 = result.append(resu, ignore_index=False)
                            result=result1
                            #print (result)
                elif (line.startswith('  ')):
                    a=0
                    d=0
                    f=0
                    s = StringIO()
                    line = line.replace('Service:', '|Servi')
                    s.write(line)
                    s.seek(0)
                    h3= pandas.read_csv(s, names = ["Servicio","Serv"], sep='|')

    #result.loc[len(df)]=dict1
resultfinal=result[['Equipo','Service','InUse','MaxUsed','Limit','Utilization']]
resultfinal

,Equipo,Service,InUse,MaxUsed,Limit,Utilization
0,ASU-ASR5K5-1,GGSN,1,1638(SundayJuly1012:34:27BOT2016),7000000,0
0,ASU-ASR5K5-1,PGW,6,4972(MondayJuly1113:47:37BOT2016),7000000,0
0,ASU-ASR5K5-1,SGW,0,6530(MondayJuly1111:46:27BOT2016),7000000,0
0,ASU-ASR5K5-1,SAEGW,0,0(Never),7000000,0
0,ASU-ASR5K5-1,Enhanced Charging Service,7,6411(MondayJuly1114:10:27BOT2016),24000000,0
0,ASU-ASR5K5-1,P2P,0,0(Never),7000000,0
0,COR-ASR5K-1,GGSN,1018905,1439122(FridayOctober1612:08:01ART2015),7000000,14
0,COR-ASR5K-1,IP Services Gateway,0,0(Never),7000000,0
0,COR-ASR5K-1,PGW,215938,228144(ThursdayJuly0712:13:11ART2016),7000000,3
0,COR-ASR5K-1,SGW,218630,230676(ThursdayJuly0712:13:11ART2016),7000000,3


In [29]:
dict1 = ["===","====","====","====","===","===","==="]
b=0
s = StringIO()
for u in device_list:
    equi=u
    h=[]
    with open(equi+'_directory hd-raid.txt') as f:
        for line in f:
            if (line.startswith('/dev/')):
                while '  ' in line:
                    line = line.replace('  ', ' ')
                s.write(line)
    s.seek(0) # "rewind" to the beginning of the StringIO object
    h= pandas.read_csv(s, names = ["Filesystem","Used", "Available","Use%","Mounted on"], sep=' ')
    with open(equi+'_show hd raid verbose.txt') as f:
        for line in f:
            if (line.startswith('  State                     :')):
                line = line.replace('  State                     : ', '')
                line = line.replace('\n', '')
                h['State']=line
            elif (line.startswith('  Degraded                  :')):
                line = line.replace('  Degraded                  : ', '')
                line = line.replace('\n', '')
                h['Degraded']=line
        
    h['Device']=equi
    if b is 0:
        b=b+1
        result=h.copy()
    else :
        result1 = result.append(h, ignore_index=True)
        result=result1
                            
resultfinal=result[['Device','Filesystem','Used', 'Available','Use%','Mounted on','State','Degraded']]
resultfinal

,Device,Filesystem,Used,Available,Use%,Mounted on,State,Degraded
0,ASU-ASR5K5-1,/dev/storage/hd-raid/disc1171742720,182624,1171560096,0%,/mnt/hd-raid,Available (clean),No
1,COR-ASR5K-1,/mnt/hd-raid/dev/storage/hd-raid/disc142508496,44101460,98407036,31%,/mnt/hd-raid,Available (clean),No
2,JO2-ASR5K5-1,/mnt/hd-raid/dev/storage/hd-raid/disc1171742720,76995068,1094747652,7%,/mnt/hd-raid,Available (active),No
3,ROS-ASR5K-1,/mnt/hd-raid/dev/storage/hd-raid/disc142508496,46311116,96197380,32%,/mnt/hd-raid,Available (clean),No
4,TOR-ASR5K5-1,/mnt/hd-raid/dev/storage/hd-raid/disc1171742720,74997256,1096745464,6%,/mnt/hd-raid,Available (active),No
5,ASUGGSN1,/mnt/hd-raid/dev/storage/hd-raid/disc142508496,15185176,127323320,11%,/mnt/hd-raid,Available (active),No
6,MONGGSN1,/mnt/hd-raid/dev/storage/hd-raid/disc142508496,7409324,135099172,5%,/mnt/hd-raid,Available (clean),No


In [122]:
#frame.a.str.contains
h=[]
dict1 = ["===","====","====","====","===","===","==="]
b=0
s = StringIO()
for u in device_list:
    equi=u
    h=[]
    i=0
    with open(equi+'_show rct stats.txt') as f:
        for line in f:
            if (line.endswith('sec\n')):
                i=i+1
                while '  ' in line:
                    line = line.replace('  ', ' ')
                s.write(line)
    s.seek(0) # "rewind" to the beginning of the StringIO object
    h= pandas.read_csv(s, names = ["Action","Type","From","To","Start Time", "Duration", "Sec"], sep=' ')
    h.insert(0, 'EQUIPO', equi)
    if i >= 5:
        h3=h.iloc[-5:]
    else:
        h3=h.iloc[-i:]
    
    h2 = h3[(h3['Sec']=='sec')]
    h1=h2[['EQUIPO','Action','Type','From','To','Start Time', 'Duration']]
    
    
    b=b+1
    if b < 2:
        #h1.loc[-1]=dict1
        result=h1.copy()
    else :
        #h1.loc[-1]=dict1
        result1 = result.append(h1, ignore_index=False)
        result=result1
result

,EQUIPO,Action,Type,From,To,Start Time,Duration
24,ASU-ASR5K5-1,Shutdown,NaN,6,0,2016-Jul-08+14:25:29.254,0.002
25,ASU-ASR5K5-1,Switchover,Planned,5,6,2016-Jul-08+14:34:12.436,17.123
26,ASU-ASR5K5-1,Shutdown,NaN,5,0,2016-Jul-08+14:38:29.163,0.002
27,ASU-ASR5K5-1,Switchover,Planned,6,5,2016-Jul-08+14:45:06.840,19.681
81,COR-ASR5K-1,Shutdown,NaN,4,0,2015-Oct-02+15:26:02.794,26.371
82,COR-ASR5K-1,Migration,Planned,16,4,2015-Oct-02+15:32:59.872,20.195
83,COR-ASR5K-1,Shutdown,NaN,16,0,2015-Oct-02+15:35:23.162,0.218
84,COR-ASR5K-1,Shutdown,NaN,13,16,2016-May-25+00:22:51.727,1.870
86,JO2-ASR5K5-1,Switchover,Planned,5,6,2016-Jul-13+00:15:21.505,17.627
87,JO2-ASR5K5-1,Switchover,Planned,6,5,2016-Jul-13+00:25:14.380,17.504


In [ ]:
b=0
dict1 = ["===","====","====","====","===","==="]
for j in device_list:
    equi=j
    path = '_show card table.txt'
    col_specification =[(0, 11), (13, 52), (53, 67), (68, 73), (75, 81),]
    col_names = ['Slot','Card Type','Oper State','SPOF','Attach']
    data = pandas.read_fwf(equi+path, colspecs=col_specification, names=col_names)
    h1=data.iloc[3:]   
    h1.insert(0, 'EQUIPO', j)
    b=b+1
    if b < 2:
        #h=pandas.DataFrame(totaline)
        #h1=h.T
        h1.loc[-1]=dict1
        result=h1.copy()
    else :
        #h=pandas.DataFrame(totaline)
        #h1=h.T
        h1.loc[-1]=dict1
        result1 = result.append(h1, ignore_index=False)
        result=result1
        
#result.columns=["Device","Version","Boot Image"]
result
filtered = result[(result['Card Type']!='None')]
filtered

In [96]:
b=0
for j in device_list:
    equi=j
    path = '_show session counters historical all.txt'
    col_specification =[(1, 4), (6, 26), (27, 37), (38, 48), (49, 59), (60, 70), (71, 81), (82, 95), (96, 109), (110,123),]
    col_names = ['Intv','Timestamp','Arrived','Rejected','Connected','Disconn','Failed','Handoffs','Renewals','CallOps']
    data = pandas.read_fwf(equi+path, colspecs=col_specification, names=col_names)
    h1=data.iloc[4:10]   
    h1.insert(0, 'EQUIPO', j)
    h1.Connected = h1.Connected.astype(np.int64)
    h1.Arrived = h1.Arrived.astype(np.int64)
    #l6=((l4/l5)*100)
    h1[equi]=(h1['Connected'] / h1['Arrived']*100)
    h2=h1[['Timestamp',equi]]
    h3=h2.T
    h4=h3[1:]
    b=b+1
    if b < 2:
        f= h1.Timestamp.tolist()
        h4.columns=f
        result=h4.copy()
    else :
        f= h1.Timestamp.tolist()
        h4.columns=f
        #h=pandas.DataFrame(totaline)
        #h1=h.T
        #h1.loc[-1]=dict1
        result1 = result.append(h4, ignore_index=False)
        result=result1
result2=result.T
result2=result2.reset_index()
result2.columns.values[0] = "Date"
result2
result3=(result2.groupby(['Date'], as_index=False).sum()).sort('Date', ascending=False)
result3.fillna('-')
result3

c:\python\lib\site-packages\pandas\core\generic.py:2698: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
c:\python\lib\site-packages\ipykernel\__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\python\lib\site-packages\ipykernel\__main__.py:34: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,Date,ASU-ASR5K5-1,COR-ASR5K-1,JO2-ASR5K5-1,ROS-ASR5K-1,TOR-ASR5K5-1,ASUGGSN1,MONGGSN1
9,2016:07:22:15:45:00,NaN,98.718571,98.219335,98.966724,98.157076,NaN,96.262927
8,2016:07:22:15:30:00,NaN,98.706332,98.172197,98.928554,98.188329,NaN,96.504987
7,2016:07:22:15:15:00,NaN,98.722798,98.169900,98.883541,98.205567,NaN,96.293243
6,2016:07:22:15:00:00,NaN,98.807406,98.189829,98.936036,98.228280,NaN,96.327720
5,2016:07:22:14:45:00,99.075501,98.561016,98.007656,98.701388,98.082874,98.930064,96.158470
4,2016:07:22:14:30:00,98.738343,98.687125,98.215863,98.940973,98.282843,99.093782,95.999589
3,2016:07:22:14:15:00,99.329333,NaN,NaN,NaN,NaN,99.265893,NaN
2,2016:07:22:14:00:00,99.633525,NaN,NaN,NaN,NaN,99.154581,NaN
1,2016:07:22:13:45:00,98.774962,NaN,NaN,NaN,NaN,98.994198,NaN
0,2016:07:22:13:30:00,99.395715,NaN,NaN,NaN,NaN,99.252356,NaN
